# Data Pipeline Template  
A repeatable pipeline for building analysis-ready datasets with validation.

## Dataset Overview  

**Describe the dataset here:**
* What does a row represent?
* What are the key variables?
* What makes this data messy/realistic?
* What is the time range?

## What this Pipeline Produces

**Artifacts**
* `data/raw/` - raw data snapshots + metadata
* `data/staged/` - parsed/normalized table (typed, missingness normalized)
* `data/warehouse/` - curated table (Parquet; optionally partitioned)
* `data/reference/validation_report.json` - contracts + anomaly rates + canaries
* `data/reference/pipeline_runs/` - run logs for reproducibility

# 0. Setup

## 0.1 Directory Structure

Create project directories for the pipeline layers

In [2]:
# Project structure setup

# Import Libraries
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timedelta, timezone
import json
import hashlib
import math

import numpy as np
import pandas as pd

from IPython.display import display

pd.set_option("display.max_columns", 180)
pd.set_option("display.width", 180)

# Define Paths
WORK_DIR = Path("work")
PROJECT_DIR = WORK_DIR / "wadi_A1"
DATA_DIR = PROJECT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
STAGED_DIR = DATA_DIR / "staged"
WH_DIR = DATA_DIR / "warehouse"
REF_DIR = DATA_DIR / "reference"
RUN_DIR = REF_DIR / "pipeline_runs"

# Create Directories
for p in [RAW_DIR, STAGED_DIR, WH_DIR, REF_DIR, RUN_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Output
print("Project:", PROJECT_DIR)
print("Raw:", RAW_DIR)
print("Staged:", STAGED_DIR)
print("Warehouse:", WH_DIR)
print("Reference:", REF_DIR)
print("Runs:", RUN_DIR)


Project: work/wadi_A1
Raw: work/wadi_A1/data/raw
Staged: work/wadi_A1/data/staged
Warehouse: work/wadi_A1/data/warehouse
Reference: work/wadi_A1/data/reference
Runs: work/wadi_A1/data/reference/pipeline_runs


## 0.2 Helper Utilities

Define reusable helper functions for the pipeline

In [5]:
# Helper Functions
class PipelineError(RuntimeError):
    pass

def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def sha16(x: str) -> str:
    return hashlib.sha256(x.encode('utf-8')).hexdigest()[:16]

def write_json(path: Path, obj: dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, default=str))

def read_json(path: Path) -> dict:
    return json.loads(path.read_text())

def require_columns(df: pd.DataFrame, cols: list[str], context: str) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise PipelineError(f'[{context}] Missing required columns: {missing}')

def require_unique(df: pd.DataFrame, key: str, context: str) -> None:
    if key not in df.columns:
        raise PipelineError(f'[{context}] Missing key column "{key}"')
    dupes = int(df[key].duplicated().sum())
    if dupes:
        raise PipelineError(f'[{context}] Key "{key}" has {dupes} duplicates')

print("Helpers ready.")


## 0.3 Configuration

Set pipeline configuration constants

In [6]:
# Pipeline Configuration
# >>> TODO:

# Dataset identifiers
# STATION_ID
# DATASET_NAME

# Run identification
# RUN_ID

# Any other constants

---

# 1. Ingest: Acquire Raw Data

Download or load raw data and save with metadata

## 1.1 Fetch Raw Data

Download or load the raw dataset

In [7]:
# Fetch Dataset

# TODO: Fetch or load raw data
# - Download from API/URL or
# - Load from local file 

# Save to RAW_DIR with RUN_ID

## 1.2 Write Raw Metadata

Document the raw data source and retrieval details

In [8]:
# Create Raw Metadata

# >>> TODO: Create and write raw metadata
# - Source URL or file path
# - Retrieval timestamp
# - File size
# - Any query parameters

---

# 2. Stage: Parse and Normalize

Convert raw data into a clean, typed DataFrame

## 2.1 Read Raw Data

**How do we read it?**

In [9]:
# Read raw data

# >>> TODO: Define read_raw_data(path) function
# Handle file format
# parse headers
# return DataFrame

## 2.2 Create Staged DataFrame

**How do we transform it?**

In [13]:
# Create staged DataFrame

# >>> TODO: Define stage_data(df_raw) function
# Create ID column
# Parse timestamps (timezone-aware UTC)
# Normalize missing values -> NaN
# Coerce to proper types
# Sort by timestamp
# Deduplicate
# Return Staged DataFrame

## 2.3 Test Staging Function

**Does it work?**

In [11]:
# Test staged function

# >>> TODO: 
# Load raw
# run staging
# display results

## 2.4 Write Staged Outputs

**How do we save it?**

In [12]:
# Write staged outputs
# >>> TODO:
# Save Parquet
# Write metadata JSON

---

# 3. Curate: Analysis-Ready Features

Add engineered features and data quality flags.

## 3.1 Create Curated DataFrame

In [14]:
# Create curated DataFrame

# >>> TODO: Define curate_data(df_staged) function
# Time Features:
# - observation_day (date only)
# - observation_hour (0-23)
# - dayofweek (0=Monday, 6=Sunday)
# - is_weekend (binary flag)
#
# Domain Specific:
# - Add flags (e.g., high_value, anomaly_indicator)
# - Add derived metrics (e.g., deltas, ratios, rolling averages)
# - Add categorical encodings if needed
#
# Return curated DataFrame


## 3.2 Test Curation Function

In [15]:
# Test Curation Function

# >>> TODO: 
# Run curation
# Display results
# Verify features

## 3.3 Choose Curated Columns and Write

In [16]:
# Choose Curated Columns
# >>> TODO: Select curated data
# 1. Define curated_columns list
# - Identifiers
# - Timestamps
# - Core measurements
# - Engineered features
# - Flags
#
# 2. Select columns: df_final = df_curated[curated_columns]

# Write to warehouse as Parquet

# Optional - Write partitioned by date

---

# 4. Validate: Contracts + Anomalies + Canaries

Define validation rules and check data quality

## 4.1 Define Contracts

**Define Rules**

In [17]:
# Define contracts

# >>> TODO: 
# Required columns (must exist and have data):
# required_cols = ['id', 'timestamp', 'kety_metric_1', ...]

# Optional columns (monitor but do not fail):
# optional_cols = ['optional_metric1', ...]
 
# Plausible ranges (for validation)
# range_checks = {
#     'metric_1': (min_val, max_val),
#     'metric_2': (min_val, max_val),
#     ....
# }


## 4.2 Implement Validation Checks

**Implement Checker**

In [18]:
# Implement validation checks

# >>> TODO: Define validate_data(df, required_cols, optional_cols, range_checks)
# Check 1. Required columns exist
# Check 2. Required columns have data ( < 99% missing)
# Check 3. Range violations (< 5% out of range)
# Check 4. Uniqueness (if applicable)
# Check 5. No future timestamps (if applicable)

# Return validation report with:
# - passed (True/False)
# - failures (list)
# - warnings (list)

## 4.3 Run Validation

**Run Checker**

In [19]:
# Validation

# >>> TODO: Call validate_data()
# print results

## 4.4 Anomaly Flags and Investigation

**Define Anomalies**

In [20]:
# Anomalies

# >>> TODO: Define create_anomaly_summary(df, range_checks)
# Calculate (but do not add to df)
# - Value anomalies (out of range counts)
# - Missingness rates by column
# - Suspicious row count

# Return summary dict

## 4.5 Run Anomaly Analysis

In [21]:
# Anomaly Analysis

# >>> TODO: Call create_anomaly_summary()
# print results

## 4.6 Canary Checks

**Define Canaries**

In [22]:
# Define Canaries

# >>> TODO: Define create_canary_summary(df)
# Group by observation_day and check:
# - Observations per day (min/median/max)
# - Drops (days with < 50% of median)
# - Overall missingness by column
# - Worst day missingness per column
# - Days with high missingness (> 30%)

# Return canary summary dict

## 4.7 Run Canary Analysis

In [23]:
# Canary Analysis
# >>> TODO: Call create_canary_summary(), 
# print results

---

# 5. Leakage Audit (Conceptual)

Document potential temporal leakage risks.

## 5.1 Write Leakage Checklist

In [1]:
# Leakage Checklist

# >>> TODO: Create leakage checklist 
# leakage_checklist = [
#   "If building rolling features, are they computed using only past data?"
#   "If you standardize/normalize, are stats computed on TRAIN only?"
#   "If you impute missing values, does the method avoid future observations?",
#   "Are you aggregating by day unavailable at prediction time?",
#   "Is prediction time defined clearly?"
#   ... add more as needed
# ]

---

# 6. Write Final Artifacts

Consolidate validation results and create run log

## 6.1 Write Validation Report

In [2]:
# Validation Report

# >>> TODO: Define write_validation_report(...)
# Consolidate: 
# - Contracts (validation_report)
# - Anomalies (anomaly_summary)
# - Canaries (canary_summary)
# - Leakage checklist

# Write to: data/reference/validation_report.json

## 6.2 Create Run Log

In [3]:
# Run log

# >>> TODO: Define create_run_log(...)
# Document:
# - run_id
# - generated_at_utc
# - inputs (raw data paths, sizes, query fingerprint)
# - outputs (staged, curated, validation paths)
# - row_definition
# - notes

# Write to: data/reference/pipeline_runs/{RUN_ID}.json

---

# 7. Self-Check and Reflection

Document insights and potential issues

## 7.1 Pipeline Reflection

In [4]:
# Pipeline reflection

# TODO: Write reflection
# reflection = [
#     "Row definition: ...",
#     "Required columns/sensors: ...",
#     "Range checks: ...",
#     "Biggest anomaly found: ...",
#     "Likely breakage scenario: ...",
#     "Temporal leakage risk: ...",
#     "Data quality insight: ...",
#     "Pipeline reproducibility: ..."
# ]

# Print formatted reflection

---

# 8. Final Summary

## Outputs Created:

## Outputs Created:

- `data/raw/` — raw data snapshot + metadata
- `data/staged/` — parsed, typed DataFrame
- `data/warehouse/` — analysis-ready curated data
- `data/reference/validation_report.json` — full validation results
- `data/reference/pipeline_runs/{run_id}.json` — execution log

## Next Steps:

1. Use curated data for analysis or modeling
2. Review validation report for data quality issues
3. Check leakage checklist before ML model development
4. Re-run pipeline with new data using same structure